In [1]:
import datetime as dt
import pandas as pd

In [2]:
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas52semanas.parquet")

In [3]:
df.sample(10)

,origin,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name
21873,False,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV3E19467 Mostrador Valladolid T1/22905,2025-08-16 14:08:59,SUCURSAL VALLADOLID
29975,False,63000.0,5.0,[BCE00601125] G601 FILTRO COMBUSTIBLE DIESEL L...,PV2E42058 Mostrador Calle 6 T2/20899,2025-08-01 18:44:01,SUCURSAL CALLE 6
107029,False,176000.0,2.0,[BCS00235125] GS235 FILTRO COMBUSTIBLE NISSAN ...,PV1E9571 Mostrador Cota T2/3039,2025-03-15 15:20:22,PRINCIPAL COTA
191988,BD41/OUT/06866,58200.0,1.0,[DCS00605189] GS605 FILTRO COMBUSTIBLE SEPARAD...,FCOT21106,2024-10-07 16:48:14,PRINCIPAL COTA
7049,False,53700.0,1.0,[DCE01002137] G1002 FILTRO COMBUSTIBLE SEPARAD...,PV1E11024 Mostrador Cota T2/4220,2025-09-10 22:05:03,PRINCIPAL COTA
182329,BD7/OUT/12474,300300.0,3.0,[BCS00603125] GS603 FILTRO COMBUSTIBLE SEPARAD...,FBUC5233,2024-10-24 19:59:38,SUCURSAL BUCARAMANGA
194596,BD6/OUT/40538,53000.0,1.0,[DAS07805025] DA7805 FILTRO AIRE SELLADO PLANT...,FBAQ11584,2024-10-01 20:57:36,SUCURSAL BARRANQUILLA
93666,BD1\OUT\58135,149100.0,3.0,[BCS00236125] GS236 FILTRO SEPARADOR COMBUSTIB...,FCOT23881,2025-04-08 17:43:34,PRINCIPAL COTA
113705,BD9/OUT/04630,82600.0,1.0,[BCS00363125] GS363 FILTRO COMBUSTIBLE 2° MACK...,FCAL3833,2025-03-05 13:57:19,SUCURSAL CALI
29492,False,60200.0,1.0,[DAB02547025] DA2547 FILTROAIRE 1_ DONSSON -K...,PV3E19298 Mostrador Valladolid T2/14924,2025-08-02 14:43:09,SUCURSAL VALLADOLID


In [4]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)




#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])

df = df.drop(columns=["origin"])

df["product_ref"] = df["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df["date_invoice"] = pd.to_datetime(df["date_invoice"])
df["mes"] = df["date_invoice"].dt.month
df["año"] = df["date_invoice"].dt.year
df["dia"] = df["date_invoice"].dt.day

In [5]:
df = df[~(df["product_ref"].isna())]

In [6]:
from datetime import datetime

# Fecha y hora actual en formato YYYYMMDD_HHMMSS
timestamp = datetime.now().strftime("%Y%m%d")

# Nombre dinámico
filename = f"/home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_{timestamp}.xlsx"

# Guardar el Excel
df.to_excel(filename, index=False)

print(f"Archivo guardado: {filename}")


Archivo guardado: /home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_20250923.xlsx
